# CODE

### Imports

In [9]:
import os
import json
import random
from datetime import datetime

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import snntorch as snn
import optuna
import tqdm.notebook as tqdm

from WOR_dataset import Wave_Order_Dataset, split_dataset, get_dataloaders
from trial_dataset import *
from WOR_plot import (plot_wave, plot_accuracies, plot_loss_curve, plot_metrics, 
                      plot_equal_prediction_values, plot_beta_values, plot_tau_values, 
                      plot_layer_weights, plot_spike_counts, plot_snn_spikes, 
                      plot_membrane_potentials, plot_threshold_potentials)
from WOR_train_val_test import train_model, validate_model, test_model

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

Device: cpu


### Hyperparameters

In [10]:
hyperparams = {
    "seed": 42,
    "std_dev": 0.0,
    "num_samples": 1000,
    "train_ratio": 0.7,
    "validation_ratio": 0.2,
    "test_ratio": 0.1,
    "freq_min": 2,
    "freq_max": 2,
    "amp_min": 1.1,
    "amp_max": 1.1,
    "offset": 1.1,
    "sample_rate": 40,       # NOTE: Change this along with freq_max to avoid aliasing.
    "duration": 3,
    "input_size": 1,
    "hidden_size": 8,
    "output_size": 2,
    "optimizer_betas": (0.99, 0.999),
    "scheduler_step_size": 30,
    "scheduler_gamma": 0.5,
    "L1_lambda": 0.001,
    "N_hidden_weights_gaussian": 30,
    "N_output_weights_gaussian": 1,
    "hidden_reset_mechanism": 'zero',
    "output_reset_mechanism": 'zero',
    "weights_hidden_min_clamped": -3.0,
    "weights_hidden_max_clamped": 3.0,
    "weights_output_min_clamped": -3.0,
    "weights_output_max_clamped": 3.0,
    #"N_hidden_tau": 0.1,
    #"N_output_tau": 0.1,
    "beta_hidden": 0.8,
    "beta_output": 0.1,
    "learn_threshold_hidden": True,
    "learn_threshold_output": True,
    "learn_beta_hidden": True,
    "learn_beta_output": True,
    "phase1": "random_uniform_0_to_2pi",
    "phase2": "random_uniform_0_to_2pi",
    "learning_rate": 0.02,
    "threshold_hidden_min": 1.0,
    "threshold_hidden_max": 1.1,
    "threshold_output_min": 1.1,
    "threshold_output_max": 1.2,
    "penalty_weight": 0.0,
    "num_epochs": 50,
    "batch_size": 32,
    "sigmoid_slope": 20,
}

### WOR Model Definition

In [11]:
class Wave_Order_Recognition_SNN(nn.Module):
    def __init__(self, hyperparams):
        super().__init__()
        hidden_size = hyperparams["hidden_size"]
        output_size = hyperparams["output_size"]
        input_size = hyperparams["input_size"]
        sample_rate = hyperparams["sample_rate"]
        duration = hyperparams["duration"]
        deltaT = 1 / sample_rate
        self.num_steps = int(duration / deltaT)
        
        # tau and beta values for hidden and output layers
        beta_hidden = torch.full((hidden_size,), hyperparams["beta_hidden"])
        beta_output = torch.full((output_size,), hyperparams["beta_output"])

        tau_hidden = -deltaT / torch.log(beta_hidden)
        tau_output = -deltaT / torch.log(beta_output)
        
        # thresholds: tuned for the hidden layer, fixed range for output
        threshold_hidden = np.random.uniform(
            hyperparams["threshold_hidden_min"],
            hyperparams["threshold_hidden_max"],
            hidden_size
        )
        threshold_output = np.random.uniform(
            hyperparams["threshold_output_min"],
            hyperparams["threshold_output_max"],
            output_size)
        
        # Gaussian initialization parameters for weights.
        # N_hidden_weights_gaussian = hyperparams["N_hidden_weights_gaussian"]
        # N_output_weights_gaussian = hyperparams["N_output_weights_gaussian"]
        # N_hidden_weights_std = np.sqrt(N_hidden_weights_gaussian)
        # N_output_weights_std = np.sqrt(N_output_weights_gaussian)
        # gaussian_mean_hidden_weights = N_hidden_weights_gaussian / sample_rate
        # gaussian_std_hidden_weights = N_hidden_weights_std / sample_rate
        # gaussian_mean_output_weights = N_output_weights_gaussian / (sample_rate * hidden_size)
        # gaussian_std_output_weights = N_output_weights_std / (sample_rate * hidden_size)
        
        self.fc1 = nn.Linear(input_size, hidden_size, bias=False)
        self.lif1 = snn.Leaky(
            beta=beta_hidden,
            threshold=threshold_hidden,
            learn_beta=hyperparams["learn_beta_hidden"],
            learn_threshold=hyperparams["learn_threshold_hidden"],
            reset_mechanism=hyperparams["hidden_reset_mechanism"],
            reset_delay=False,
            spike_grad= snn.surrogate.fast_sigmoid(slope=hyperparams["sigmoid_slope"])
        )
        self.fc2 = nn.Linear(hidden_size, output_size, bias=False)
        self.lif2 = snn.Leaky(
            beta=beta_output,
            threshold=threshold_output,
            learn_beta=hyperparams["learn_beta_output"],
            learn_threshold=hyperparams["learn_threshold_output"],
            reset_mechanism=hyperparams["output_reset_mechanism"],
            reset_delay=False,
            spike_grad= snn.surrogate.fast_sigmoid(slope=hyperparams["sigmoid_slope"])

        )
        # For Gaussian initialization
        # self._initialize_weights(
        #     gaussian_mean_hidden_weights, gaussian_std_hidden_weights,
        #     gaussian_mean_output_weights, gaussian_std_output_weights
        # )
        # For Xavier initialization
        self._initialize_weights()
    
    # Normal initialization
    # def _initialize_weights(self, mean_hidden, std_hidden, mean_output, std_output):
    #     nn.init.normal_(self.fc1.weight, mean=mean_hidden, std=std_hidden)
    #     nn.init.normal_(self.fc2.weight, mean=mean_output, std=std_output)

    # Xavier initialization
    def _initialize_weights(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)

    
    def forward(self, x, mem1=None, mem2=None):
        batch_size = x.size(0)
        if mem1 is None:
            mem1 = torch.zeros(batch_size, self.fc1.out_features, device=x.device)
        if mem2 is None:
            mem2 = torch.zeros(batch_size, self.fc2.out_features, device=x.device)
        
        spk1_rec, mem1_rec, spk2_rec, mem2_rec = [], [], [], []
        hidden_spike_count = 0
        output_spike_count = 0
        
        for step in range(self.num_steps):
            cur1 = self.fc1(x[:, step].unsqueeze(1))
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            
            spk1_rec.append(spk1)
            mem1_rec.append(mem1)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)
            
            hidden_spike_count += spk1.sum().item()
            output_spike_count += spk2.sum().item()
        
        return (torch.stack(spk1_rec, dim=0),
                torch.stack(mem1_rec, dim=0),
                torch.stack(spk2_rec, dim=0),
                torch.stack(mem2_rec, dim=0),
                hidden_spike_count,
                output_spike_count)

### Optuna Objective for HPO

In [12]:
def objective(trial):
    # Override a few hyperparameters using trial suggestions.
    #hyperparams["learning_rate"] = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    #hyperparams["sigmoid_slope"] = trial.suggest_float("sigmoid_slope", 3, 30)
    hyperparams["threshold_hidden_min"] = trial.suggest_float("threshold_hidden_min", 0.3, 0.5)
    hyperparams["threshold_hidden_max"] = trial.suggest_float("threshold_hidden_max", 2.9, 3)
    hyperparams["beta_hidden"] = trial.suggest_float("beta_hidden", 0.7, 0.9)
    hyperparams["threshold_output_min"] = trial.suggest_float("threshold_output_min", 0.3, 0.5)
    hyperparams["threshold_output_max"] = trial.suggest_float("threshold_output_max", 8, 10)
    hyperparams["beta_output"] = trial.suggest_float("beta_output", 0.01, 0.4)
    # # Create dataset and dataloaders.
    # dataset = Wave_Order_Dataset(
    #     hyperparams["num_samples"],
    #     hyperparams["sample_rate"],
    #     hyperparams["duration"],
    #     hyperparams["freq_min"],
    #     hyperparams["freq_max"],
    #     hyperparams["amp_min"],
    #     hyperparams["amp_max"],
    #     hyperparams["std_dev"],
    #     hyperparams["offset"]
    # )

    dataset = ConcatenatedWaveDataset(
        num_samples=hyperparams["num_samples"],
        sample_rate=hyperparams["sample_rate"],
        duration=hyperparams["duration"],  # duration of each wave segment
        frequency=2.0,
        amplitude=1.0,
        std_dev=hyperparams["std_dev"],
        offset=hyperparams["offset"]
    )


    train_dataset, validation_dataset, _ = split_dataset(
        dataset,
        hyperparams["train_ratio"],
        hyperparams["validation_ratio"],
        hyperparams["test_ratio"]
    )
    train_loader, validation_loader, _ = get_dataloaders(
        train_dataset, validation_dataset, [], hyperparams["batch_size"]
    )
    
    # Instantiate model, loss, optimizer, and scheduler.
    model = Wave_Order_Recognition_SNN(hyperparams).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adamax(model.parameters(), lr=hyperparams["learning_rate"],
                                   betas=hyperparams["optimizer_betas"])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=hyperparams["scheduler_step_size"],
                                                gamma=hyperparams["scheduler_gamma"])
    
    # Run a short training loop for HPO (using fewer epochs for speed).
    num_epochs = hyperparams["num_epochs"] // 10
    for epoch in range(num_epochs):
        train_loss, _, _, _, _, _, _= train_model(model, train_loader, criterion, optimizer,
                                             epoch, num_epochs, hyperparams["batch_size"],
                                             hyperparams["hidden_size"], hyperparams["output_size"],
                                             hyperparams["weights_hidden_min_clamped"],
                                             hyperparams["weights_hidden_max_clamped"],
                                             hyperparams["weights_output_min_clamped"],
                                             hyperparams["weights_output_max_clamped"],
                                             hyperparams["penalty_weight"],
                                             hyperparams["L1_lambda"], device)
        val_accuracy, val_loss = validate_model(model, validation_loader, criterion, device)
        scheduler.step()
        if epoch % 5 == 0:
            print(f"Trial {trial.number} Epoch {epoch}: Val Acc {val_accuracy:.2f}% Loss {val_loss:.4f}")
    
    # validation accuracy (metric to maximize)
    return val_accuracy

### Full Training Run (after HPO)

In [13]:
def run_training(hyperparams):
    set_seed(hyperparams["seed"])
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")
    
    # Folder for saving outputs.
    now = datetime.now()
    output_folder = os.path.join(os.getcwd(), "thesis_simulations", now.strftime("run_%Y%m%d_%H%M%S"))
    os.makedirs(output_folder, exist_ok=True)
    
    # dataset = Wave_Order_Dataset(
    #     hyperparams["num_samples"],
    #     hyperparams["sample_rate"],
    #     hyperparams["duration"],
    #     hyperparams["freq_min"],
    #     hyperparams["freq_max"],
    #     hyperparams["amp_min"],
    #     hyperparams["amp_max"],
    #     hyperparams["std_dev"],
    #     hyperparams["offset"]
    # )

    dataset = ConcatenatedWaveDataset(
        num_samples=hyperparams["num_samples"],
        sample_rate=hyperparams["sample_rate"],
        duration=hyperparams["duration"],  # duration of each wave segment
        frequency=2.0,
        amplitude=1.0,
        std_dev=hyperparams["std_dev"],
        offset=hyperparams["offset"]
    )



    train_dataset, validation_dataset, test_dataset = split_dataset(
        dataset,
        hyperparams["train_ratio"],
        hyperparams["validation_ratio"],
        hyperparams["test_ratio"]
    )
    train_loader, validation_loader, test_loader = get_dataloaders(
        train_dataset, validation_dataset, test_dataset, hyperparams["batch_size"]
    )
    
    # Instantiate model, loss, optimizer, and scheduler.
    model = Wave_Order_Recognition_SNN(hyperparams).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adamax(model.parameters(), lr=hyperparams["learning_rate"],
                                   betas=hyperparams["optimizer_betas"])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=hyperparams["scheduler_step_size"],
                                                gamma=hyperparams["scheduler_gamma"])
    
    # Lists for recording metrics.
    train_acc_list = []
    train_loss_list = []
    val_acc_list = []
    val_loss_list = []
    equal_prediction_values = []  # NEW list for equal prediction count
    beta1_values = []
    beta2_values = []
    weights_hidden_layer = []
    weights_output_layer = []
    threshold_hidden_layer = []
    threshold_output_layer = []
    hidden_spike_count_list = []
    output_spike_count_list = []
    output_spike_counts_neuron0_list = []
    output_spike_counts_neuron1_list = []
    
    num_epochs = hyperparams["num_epochs"]
    for epoch in range(num_epochs):
        (train_loss, avg_hidden_spike, avg_output_spike, avg_out0,
         avg_out1, avg_equal_pred, avg_train_accuracy) = train_model(model, train_loader, criterion, optimizer,
                                                 epoch, num_epochs, hyperparams["batch_size"],
                                                 hyperparams["hidden_size"], hyperparams["output_size"],
                                                 hyperparams["weights_hidden_min_clamped"],
                                                 hyperparams["weights_hidden_max_clamped"],
                                                 hyperparams["weights_output_min_clamped"],
                                                 hyperparams["weights_output_max_clamped"],
                                                 hyperparams["penalty_weight"],
                                                 hyperparams["L1_lambda"], device)
        val_accuracy, val_loss = validate_model(model, validation_loader, criterion, device)
        scheduler.step()
        
        train_loss_list.append(train_loss)
        train_acc_list.append(avg_train_accuracy)
        val_acc_list.append(val_accuracy)
        val_loss_list.append(val_loss)
        equal_prediction_values.append(avg_equal_pred)
        beta1_values.append(model.lif1.beta.detach().cpu().numpy().flatten())
        beta2_values.append(model.lif2.beta.detach().cpu().numpy().flatten())
        weights_hidden_layer.append(model.fc1.weight.detach().cpu().numpy().copy())
        weights_output_layer.append(model.fc2.weight.detach().cpu().numpy().copy())
        threshold_hidden_layer.append(model.lif1.threshold.detach().cpu().numpy().flatten())
        threshold_output_layer.append(model.lif2.threshold.detach().cpu().numpy().flatten())
        hidden_spike_count_list.append(avg_hidden_spike)
        output_spike_count_list.append(avg_output_spike)
        output_spike_counts_neuron0_list.append(avg_out0)
        output_spike_counts_neuron1_list.append(avg_out1)
        
        print(f"Epoch {epoch+1}/{num_epochs}: Val Acc = {val_accuracy:.2f}% | Val Loss = {val_loss:.4f}")
    
    # Evaluate on test data.
    test_metrics = test_model(model, test_loader, criterion, device)
    print("Test Metrics:")
    print(test_metrics)
    
    # Save final model.
    model_save_path = os.path.join(output_folder, "final_model.pth")
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")
    
    # Return dictionary with all recorded metrics.
    return {
        "output_folder": output_folder,
        "train_loader": train_loader,
        "validation_loader": validation_loader,
        "test_loader": test_loader,
        "model": model,
        "test_metrics": test_metrics,
        "num_epochs": num_epochs,
        "deltaT": 1 / hyperparams["sample_rate"],
        "train_loss_list": train_loss_list,
        "val_acc_list": val_acc_list,
        "val_loss_list": val_loss_list,
        "equal_prediction_values": equal_prediction_values,
        "beta1_values": beta1_values,
        "beta2_values": beta2_values,
        "weights_hidden_layer": weights_hidden_layer,
        "weights_output_layer": weights_output_layer,
        "threshold_hidden_layer": threshold_hidden_layer,
        "threshold_output_layer": threshold_output_layer,
        "hidden_spike_count_list": hidden_spike_count_list,
        "output_spike_count_list": output_spike_count_list,
        "output_spike_counts_neuron0_list": output_spike_counts_neuron0_list,
        "output_spike_counts_neuron1_list": output_spike_counts_neuron1_list,
    }

### 'mode' Selection and Execution

In [14]:
# mode = "train" or "hpo"
mode = "hpo"

if mode == "hpo":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    best_trial = study.best_trial
    print("Best trial:")
    print(f"  Value: {best_trial.value}")
    print("  Params:")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")
    with open("best_hyperparameters.json", "w") as f:
        json.dump(best_trial.params, f, indent=4)
    print("Best hyperparameters saved to best_hyperparameters.json")
    
elif mode == "train":
    with open("best_hyperparameters.json", "r") as f:
        best_tuned = json.load(f)
    final_hyperparams = dict(hyperparams)
    final_hyperparams.update(best_tuned)
    results = run_training(final_hyperparams)


[I 2025-02-18 15:56:29,909] A new study created in memory with name: no-name-14c76b43-3f89-46e4-baf1-aaa80df8477b


Trial 0 Epoch 0: Val Acc 54.50% Loss 0.6931


[I 2025-02-18 15:56:35,572] Trial 0 finished with value: 52.0 and parameters: {'threshold_hidden_min': 0.4555448339732601, 'threshold_hidden_max': 2.990079160717714, 'beta_hidden': 0.769333644372323, 'threshold_output_min': 0.35100843783930713, 'threshold_output_max': 8.986913365244511, 'beta_output': 0.2301136991768539}. Best is trial 0 with value: 52.0.


Trial 1 Epoch 0: Val Acc 47.50% Loss 0.6931


[I 2025-02-18 15:56:41,041] Trial 1 finished with value: 74.5 and parameters: {'threshold_hidden_min': 0.48981136519815655, 'threshold_hidden_max': 2.9577196055056723, 'beta_hidden': 0.7811638741654722, 'threshold_output_min': 0.3690001879918385, 'threshold_output_max': 8.92732003149354, 'beta_output': 0.015532513718298394}. Best is trial 1 with value: 74.5.


...

Trial 48 Epoch 0: Val Acc 43.50% Loss 0.6931


[I 2025-02-18 16:00:54,162] Trial 48 finished with value: 49.5 and parameters: {'threshold_hidden_min': 0.46300817199569894, 'threshold_hidden_max': 2.967898165892902, 'beta_hidden': 0.8804530544449064, 'threshold_output_min': 0.47201683960071156, 'threshold_output_max': 8.142235073334856, 'beta_output': 0.26891367973801567}. Best is trial 34 with value: 100.0.


Trial 49 Epoch 0: Val Acc 49.50% Loss 0.6931


[I 2025-02-18 16:00:59,583] Trial 49 finished with value: 50.5 and parameters: {'threshold_hidden_min': 0.44592201387506053, 'threshold_hidden_max': 2.951868099104198, 'beta_hidden': 0.8926647832868359, 'threshold_output_min': 0.4898265447213076, 'threshold_output_max': 8.01996138336676, 'beta_output': 0.31578457046215597}. Best is trial 34 with value: 100.0.


Best trial:
  Value: 100.0
  Params:
    threshold_hidden_min: 0.46438149062801726
    threshold_hidden_max: 2.9646876910192077
    beta_hidden: 0.8468230841059797
    threshold_output_min: 0.4436440954754969
    threshold_output_max: 9.36989913401044
    beta_output: 0.2719411200662194
Best hyperparameters saved to best_hyperparameters.json


# PLOTS - NEW

In [ ]:
# PLOTS (Run only after a full training run)
unique_folder_name = results["output_folder"]
num_epochs = results["num_epochs"]
deltaT = results["deltaT"]
model = results["model"]

train_loader = results["train_loader"]
test_loader = results["test_loader"]
test_metrics = results["test_metrics"]

# Sample Dataset plot
plot_wave(train_loader, save_path=os.path.join(unique_folder_name, 'wave_samples.png'))

plot_accuracies(num_epochs, train_accuracy=results["train_acc_list"], validation_accuracy=results["val_acc_list"],
                save_path=os.path.join(unique_folder_name, 'accuracy_plot'))


# Loss curve plot (using recorded training and validation losses)
plot_loss_curve(results["train_loss_list"], results["val_loss_list"], num_epochs,
                os.path.join(unique_folder_name, 'loss_curve'))

# Confusion Matrix plot using test metrics.
plot_metrics(test_metrics, os.path.join(unique_folder_name, 'evaluation_plots'))

# Equal Prediction Count plot.
plot_equal_prediction_values(results["equal_prediction_values"], num_epochs,
                             os.path.join(unique_folder_name, 'equal_prediction_values'))

# Beta Values plots for hidden and output layers.
plot_beta_values(results["beta1_values"], num_epochs,
                 os.path.join(unique_folder_name, 'beta_hidden_layer'), layer_name='Hidden')
plot_beta_values(results["beta2_values"], num_epochs,
                 os.path.join(unique_folder_name, 'beta_output_layer'), layer_name='Output')

# Tau Values plots (derived from beta values and deltaT).
plot_tau_values(results["beta1_values"], num_epochs, deltaT,
                os.path.join(unique_folder_name, 'tau_hidden_layer'), layer_name='Hidden')
plot_tau_values(results["beta2_values"], num_epochs, deltaT,
                os.path.join(unique_folder_name, 'tau_output_layer'), layer_name='Output')

# Weight Distribution plots for hidden and output layers.
plot_layer_weights(results["weights_hidden_layer"], num_epochs,
                   os.path.join(unique_folder_name, 'weights_hidden_layer'), layer_name='Hidden')
plot_layer_weights(results["weights_output_layer"], num_epochs,
                   os.path.join(unique_folder_name, 'weights_output_layer'), layer_name='Output')

# Spike Count vs Epochs plot.
plot_spike_counts(results["hidden_spike_count_list"], results["output_spike_count_list"],
                  results["output_spike_counts_neuron0_list"], results["output_spike_counts_neuron1_list"],
                  num_epochs, os.path.join(unique_folder_name, 'spike_counts'))

# Spikes plots for hidden and output layers.
plot_snn_spikes(model, test_loader, device, os.path.join(unique_folder_name, 'hidden_layer_spikes'),
                layer_name='Hidden', layer_size=hyperparams["hidden_size"], num_steps=model.num_steps)
plot_snn_spikes(model, test_loader, device, os.path.join(unique_folder_name, 'output_layer_spikes'),
                layer_name='Output', layer_size=hyperparams["output_size"], num_steps=model.num_steps)

# Membrane Potential plots for hidden and output layers.
plot_membrane_potentials(model, test_loader, device, 'Hidden', hyperparams["hidden_size"], model.num_steps,
                         os.path.join(unique_folder_name, 'hidden_membrane_potentials'))
plot_membrane_potentials(model, test_loader, device, 'Output', hyperparams["output_size"], model.num_steps,
                         os.path.join(unique_folder_name, 'output_membrane_potentials'))

# Threshold Potentials plots for hidden and output layers.
plot_threshold_potentials(results["threshold_hidden_layer"], num_epochs,
                          os.path.join(unique_folder_name, 'threshold_hidden_layer'), 'Hidden')
plot_threshold_potentials(results["threshold_output_layer"], num_epochs,
                          os.path.join(unique_folder_name, 'threshold_output_layer'), 'Output')

# PLOTS - OLD

ONLY AFTER HAVING DONE AT LEAST 1 FULL TRAINING RUN

In [14]:
unique_folder_name = results["output_folder"]
num_epochs = results["num_epochs"]
deltaT = results["deltaT"]
model = results["model"]

train_loader = results["train_loader"]
test_loader = results["test_loader"]
test_metrics = results["test_metrics"]

### Sample Dataset

In [ ]:
plot_wave(train_loader, save_path=os.path.join(unique_folder_name, 'wave_samples.png'))

### Accuracy

In [ ]:
plot_accuracies(num_epochs, test_accuracies, validation_accuracies, os.path.join(unique_folder_name, 'accuracy_plot'))

### Loss

In [ ]:
plot_loss_curve(loss_hist, test_loss_hist, num_epochs, os.path.join(unique_folder_name, 'loss_curve'))

### Confusion Matrix

In [ ]:
plot_metrics(test_metrics,os.path.join(unique_folder_name, 'evaluation_plots'))

### Equal Prediction Count

In [ ]:
plot_equal_prediction_values(equal_prediction_values, num_epochs, os.path.join(unique_folder_name, 'equal_prediction_values'))

### Beta Hidden Layer

In [ ]:
plot_beta_values(beta1_values, num_epochs, os.path.join(unique_folder_name, 'beta_hidden_layer'), layer_name='Hidden')

### Beta Output Layer

In [ ]:
plot_beta_values(beta2_values, num_epochs, os.path.join(unique_folder_name, 'beta_output_layer'), layer_name='Output')

### Tau Hidden Layer

In [ ]:
plot_tau_values(beta1_values, num_epochs, deltaT, os.path.join(unique_folder_name, 'tau_hidden_layer'), layer_name='Hidden')

### Tau Output Layer

In [ ]:
plot_tau_values(beta2_values, num_epochs, deltaT, os.path.join(unique_folder_name, 'tau_output_layer'), layer_name='Output')

### Weights Hidden Layer

In [ ]:
plot_layer_weights(weights_hidden_layer, num_epochs, os.path.join(unique_folder_name, 'weights_hidden_layer'), layer_name='Hidden')

### Weights Output Layer

In [ ]:
plot_layer_weights(weights_output_layer, num_epochs, os.path.join(unique_folder_name, 'weights_output_layer'), layer_name='Output')

### Spike Count vs Epochs

In [ ]:
plot_spike_counts(hidden_spike_count, output_spike_count, output_spike_counts_neuron0, output_spike_counts_neuron1, num_epochs, os.path.join(unique_folder_name, 'spike_counts'))

### Spikes Hidden Layer

In [ ]:
plot_snn_spikes(model, test_loader, device, os.path.join(unique_folder_name, 'hidden_layer_spikes'), layer_name='Hidden', layer_size=hidden_size, num_steps=num_steps)

### Spikes Output Layer

In [ ]:
plot_snn_spikes(model, test_loader, device, os.path.join(unique_folder_name, 'output_layer_spikes'), layer_name='Output', layer_size=output_size, num_steps=num_steps)

### Vmem Hidden Layer

In [ ]:
plot_membrane_potentials(model, test_loader, device, 'Hidden', hidden_size, num_steps, os.path.join(unique_folder_name, 'hidden_membrane_potentials'))

### Vmem Output Layer

In [ ]:
plot_membrane_potentials(model, test_loader, device, 'Output', output_size, num_steps, os.path.join(unique_folder_name, 'output_membrane_potentials'))

### Threshold Hidden Layer

In [ ]:
plot_threshold_potentials(threshold_hidden_layer, num_epochs, os.path.join(unique_folder_name, 'threshold_hidden_layer'), 'Hidden')

### Threshold Output Layer

In [ ]:
plot_threshold_potentials(threshold_output_layer, num_epochs, os.path.join(unique_folder_name, 'threshold_output_layer'), 'Output')